In [1]:
import os
import sys
sys.path.append(os.path.abspath(".."))

In [2]:
import torch
import numpy as np
import torchvision.transforms as transforms
import seaborn as sns
import ristretto.activations as ra
import ristretto.models as rm
import ristretto.utils as ru
import pandas as pd
import warnings
from functools import partial

with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    import ray
    from ray import tune
    from ray.tune import JupyterNotebookReporter
    from ray.tune.schedulers import ASHAScheduler
# from torch.utils.tensorboard import SummaryWriter

In [3]:
# set pytorch precision
torch.set_default_tensor_type(torch.FloatTensor)
torch.set_default_dtype(torch.bfloat16)

In [4]:
ray.init(runtime_env={"working_dir": ".."}, log_to_driver=False)
# ray.init(runtime_env={"working_dir": ".."})

2022-12-21 00:15:21,181	INFO worker.py:1528 -- Started a local Ray instance.
2022-12-21 00:15:22,113	INFO packaging.py:527 -- Creating a file package for local directory '..'.
2022-12-21 00:15:22,494	INFO packaging.py:354 -- Pushing file package 'gcs://_ray_pkg_d6e534b5664b4b09.zip' (8.77MiB) to Ray cluster...
2022-12-21 00:15:22,631	INFO packaging.py:367 -- Successfully pushed file package 'gcs://_ray_pkg_d6e534b5664b4b09.zip'.


Python version:,3.9.13
Ray version:,2.1.0


### Including ReLU6 subgradients as hyperparameters

In [5]:
config = {
    "batch_size": tune.choice([32, 64, 128]),
    "lr": tune.loguniform(1e-4, 1e-2),
    "alpha": tune.uniform(0, 1),
    "beta": tune.uniform(0, 1),
}

In [6]:
epochs = 10
scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=epochs,
    grace_period=3,
    reduction_factor=2)
reporter = JupyterNotebookReporter(
    # parameter_columns=["l1", "l2", "lr", "batch_size"],
    metric_columns=["loss", "accuracy", "training_iteration"])

In [7]:
result_sub = tune.run(
    partial(ru.tune_train_fullyconnected),
    resources_per_trial={"cpu": 8, "gpu": 1},
    config=config,
    num_samples=12,
    scheduler=scheduler,
    progress_reporter=reporter,
    fail_fast="raise"
)

C:\Users\tomma\AppData\Roaming\Python\Python39\site-packages\ray\tune\trainable\function_trainable.py:609: DeprecationWarning: `checkpoint_dir` in `func(config, checkpoint_dir)` is being deprecated. To save and load checkpoint in trainable functions, please use the `ray.air.session` API:

from ray.air import session

def train(config):
    # ...
    session.report({"metric": metric}, checkpoint=checkpoint)

For more information please see https://docs.ray.io/en/master/tune/api_docs/trainable.html

  warnings.warn(
C:\Users\tomma\AppData\Roaming\Python\Python39\site-packages\ray\tune\execution\trial_runner.py:363: UserWarning: fail_fast='raise' detected. Be careful when using this mode as resources (such as Ray processes, file descriptors, and temporary files) may not be cleaned up properly. To use a safer mode, use fail_fast=True.
  warnings.warn(


Trial name,accuracy,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
tune_train_fullyconnected_2dc45_00000,97.0833,2022-12-21_00-23-59,True,,a39cafdf98ba4fbaa97c431318d6aa2f,MSI-Tommy,10,0.11024,127.0.0.1,8184,True,505.671,49.7499,505.671,1671578639,0,,10,2dc45_00000,0.00597143
tune_train_fullyconnected_2dc45_00001,96.475,2022-12-21_00-26-50,True,,bc5f2b75c8464731a0d373b590e50c5a,MSI-Tommy,3,0.13421,127.0.0.1,12524,True,164.871,53.0305,164.871,1671578810,0,,3,2dc45_00001,0.00711083
tune_train_fullyconnected_2dc45_00002,97.975,2022-12-21_00-35-23,True,,bf35c64491664f47bc48055e6d37e9a7,MSI-Tommy,10,0.072265,127.0.0.1,22996,True,506.348,49.7854,506.348,1671579323,0,,10,2dc45_00002,0
tune_train_fullyconnected_2dc45_00003,97.925,2022-12-21_00-43-54,True,,8f55dab6fe7e47a68d2988734d6aaf2e,MSI-Tommy,10,0.0655925,127.0.0.1,22296,True,502.787,50.101,502.787,1671579834,0,,10,2dc45_00003,0.0198715
tune_train_fullyconnected_2dc45_00004,96.025,2022-12-21_00-46-23,True,,7e2630411f7343cd824ec8189966bffe,MSI-Tommy,3,0.135981,127.0.0.1,2896,True,141.171,44.6457,141.171,1671579983,0,,3,2dc45_00004,0.0197275
tune_train_fullyconnected_2dc45_00005,92.8917,2022-12-21_00-49-11,True,,315f2ac973a34cbb95a6dcecf1149b5c,MSI-Tommy,3,0.254888,127.0.0.1,6120,True,161.831,55.6547,161.831,1671580151,0,,3,2dc45_00005,0.00605392
tune_train_fullyconnected_2dc45_00006,97.25,2022-12-21_00-54-25,True,,cca467772b26413d89f8114675408de7,MSI-Tommy,6,0.0892789,127.0.0.1,17424,True,305.077,56.0593,305.077,1671580465,0,,6,2dc45_00006,0.00597739
tune_train_fullyconnected_2dc45_00007,96.175,2022-12-21_00-57-48,True,,7d4639a1021041eba9793a4d73b00afb,MSI-Tommy,3,0.140425,127.0.0.1,14376,True,193.981,63.1758,193.981,1671580668,0,,3,2dc45_00007,0.0207176
tune_train_fullyconnected_2dc45_00008,97.8917,2022-12-21_01-08-33,True,,6811072a1d5b435c8e81cbd22c90a670,MSI-Tommy,10,0.0712869,127.0.0.1,20748,True,635.343,62.6541,635.343,1671581313,0,,10,2dc45_00008,0.00372744
tune_train_fullyconnected_2dc45_00009,94.5917,2022-12-21_01-11-42,True,,bc19924873184c9982d188e0dc03398b,MSI-Tommy,3,0.192178,127.0.0.1,21876,True,180.796,57.8745,180.796,1671581502,0,,3,2dc45_00009,0


2022-12-21 01:26:00,439	INFO tune.py:777 -- Total run time: 4237.60 seconds (4237.39 seconds for the tuning loop).


In [8]:
result_sub.results_df.to_csv("results_with_subgradient_fc.csv")

In [9]:
result_sub.results_df

,loss,accuracy,time_this_iter_s,should_checkpoint,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,...,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,warmup_time,experiment_tag,config/batch_size,config/lr,config/alpha,config/beta
trial_id,,,,,,,,,,,,,,,,,,,,,
2dc45_00000,0.110240,97.083333,49.749876,True,True,None,None,10,a39cafdf98ba4fbaa97c431318d6aa2f,2022-12-21_00-23-59,...,127.0.0.1,505.670660,0,10,0.005971,"0_alpha=0.8138,batch_size=64,beta=0.1880,lr=0....",64,0.000496,0.813764,0.188042
2dc45_00001,0.134210,96.475000,53.030455,True,True,None,None,3,bc5f2b75c8464731a0d373b590e50c5a,2022-12-21_00-26-50,...,127.0.0.1,164.870803,0,3,0.007111,"1_alpha=0.6199,batch_size=32,beta=0.2280,lr=0....",32,0.005694,0.619898,0.227990
2dc45_00002,0.072265,97.975000,49.785440,True,True,None,None,10,bf35c64491664f47bc48055e6d37e9a7,2022-12-21_00-35-23,...,127.0.0.1,506.348066,0,10,0.000000,"2_alpha=0.3724,batch_size=32,beta=0.3181,lr=0....",32,0.000965,0.372381,0.318065
2dc45_00003,0.065593,97.925000,50.101012,True,True,None,None,10,8f55dab6fe7e47a68d2988734d6aaf2e,2022-12-21_00-43-54,...,127.0.0.1,502.786909,0,10,0.019871,"3_alpha=0.6086,batch_size=32,beta=0.0274,lr=0....",32,0.001645,0.608594,0.027391
2dc45_00004,0.135981,96.025000,44.645682,True,True,None,None,3,7e2630411f7343cd824ec8189966bffe,2022-12-21_00-46-23,...,127.0.0.1,141.171358,0,3,0.019727,"4_alpha=0.6845,batch_size=128,beta=0.6872,lr=0...",128,0.000489,0.684474,0.687217
2dc45_00005,0.254888,92.891667,55.654669,True,True,None,None,3,315f2ac973a34cbb95a6dcecf1149b5c,2022-12-21_00-49-11,...,127.0.0.1,161.830748,0,3,0.006054,"5_alpha=0.0237,batch_size=64,beta=0.2491,lr=0....",64,0.000185,0.023730,0.249123
2dc45_00006,0.089279,97.250000,56.059329,True,True,None,None,6,cca467772b26413d89f8114675408de7,2022-12-21_00-54-25,...,127.0.0.1,305.076777,0,6,0.005977,"6_alpha=0.8993,batch_size=128,beta=0.0841,lr=0...",128,0.000761,0.899314,0.084068
2dc45_00007,0.140425,96.175000,63.175789,True,True,None,None,3,7d4639a1021041eba9793a4d73b00afb,2022-12-21_00-57-48,...,127.0.0.1,193.981206,0,3,0.020718,"7_alpha=0.9224,batch_size=32,beta=0.2551,lr=0....",32,0.000436,0.922377,0.255105
2dc45_00008,0.071287,97.891667,62.654132,True,True,None,None,10,6811072a1d5b435c8e81cbd22c90a670,2022-12-21_01-08-33,...,127.0.0.1,635.343281,0,10,0.003727,"8_alpha=0.7206,batch_size=32,beta=0.8130,lr=0....",32,0.001570,0.720565,0.812957


In [10]:
print(f"""Best loss: {result_sub.get_best_config(metric="loss", mode="min")}""")
print(f"""Best accuracy: {result_sub.get_best_config(metric="accuracy", mode="max")}""")

Best loss: {'batch_size': 32, 'lr': 0.0016451491753928277, 'alpha': 0.608593524621869, 'beta': 0.02739124619724076}
Best accuracy: {'batch_size': 32, 'lr': 0.0009654658372955936, 'alpha': 0.37238068772818855, 'beta': 0.31806467118888126}


### With fixed ReLU6 subgradients

In [14]:
config = {
    "batch_size": tune.choice([32, 64, 128]),
    "lr": tune.loguniform(1e-4, 1e-2),
    "alpha": tune.choice([0]),
    "beta": tune.choice([0]),
}

In [15]:
epochs = 10
scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=epochs,
    grace_period=3,
    reduction_factor=2)
reporter = JupyterNotebookReporter(
    # parameter_columns=["l1", "l2", "lr", "batch_size"],
    metric_columns=["loss", "accuracy", "training_iteration"])

In [16]:
result_no_sub = tune.run(
    partial(ru.tune_train_fullyconnected),
    resources_per_trial={"cpu": 8, "gpu": 1},
    config=config,
    num_samples=12,
    scheduler=scheduler,
    progress_reporter=reporter
)

Trial name,accuracy,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
tune_train_fullyconnected_c2668_00000,93.325,2022-12-21_11-08-31,True,,493ad909e1a14ba2a4cedca9690ab11d,MSI-Tommy,10,0.246523,127.0.0.1,7412,True,701.854,80.5819,701.854,1671617311,0,,10,c2668_00000,0.00690222
tune_train_fullyconnected_c2668_00001,94.2083,2022-12-21_11-19-47,True,,19d824473577498395a6bb78a75b344f,MSI-Tommy,10,0.216098,127.0.0.1,2636,True,666.734,62.5058,666.734,1671617987,0,,10,c2668_00001,0.00807309
tune_train_fullyconnected_c2668_00002,91.7417,2022-12-21_11-22-41,True,,d65a19d5110440de8dc50697b977a161,MSI-Tommy,3,0.301238,127.0.0.1,6688,True,163.749,53.8684,163.749,1671618161,0,,3,c2668_00002,0.00603676
tune_train_fullyconnected_c2668_00003,97.475,2022-12-21_11-31-25,True,,c3c0ab5f4f1644d7ba21ed43fe2f843e,MSI-Tommy,10,0.0806567,127.0.0.1,10984,True,517.889,51.8304,517.889,1671618685,0,,10,c2668_00003,0
tune_train_fullyconnected_c2668_00004,98.0917,2022-12-21_11-41-18,True,,1347882288004c0d9d9d18bca1d9d85d,MSI-Tommy,10,0.0763712,127.0.0.1,13280,True,585.821,69.1014,585.821,1671619278,0,,10,c2668_00004,0.0101876
tune_train_fullyconnected_c2668_00005,95.2,2022-12-21_11-46-57,True,,54d8be19f0ec4c92be41f1d34f97f90d,MSI-Tommy,6,0.179147,127.0.0.1,6692,True,329.252,56.3268,329.252,1671619617,0,,6,c2668_00005,0.0126114
tune_train_fullyconnected_c2668_00006,95.5333,2022-12-21_11-57-10,True,,379f30c73abb423c99c21cecc51e5e1c,MSI-Tommy,10,0.166914,127.0.0.1,12524,True,604.499,58.147,604.499,1671620230,0,,10,c2668_00006,0
tune_train_fullyconnected_c2668_00007,97.2583,2022-12-21_12-07-42,True,,3be13458a63c43c6be134d20882ed7b3,MSI-Tommy,10,0.0965037,127.0.0.1,11568,True,624.285,53.4621,624.285,1671620862,0,,10,c2668_00007,0.00402331
tune_train_fullyconnected_c2668_00008,97.925,2022-12-21_12-17-07,True,,0759a56640ea439499d397c24257091e,MSI-Tommy,10,0.0703743,127.0.0.1,26444,True,557.804,52.4268,557.804,1671621427,0,,10,c2668_00008,0
tune_train_fullyconnected_c2668_00009,98.15,2022-12-21_12-28-13,True,,7405bee9275a4ef19b79f098f3d6d8dd,MSI-Tommy,10,0.0630877,127.0.0.1,27744,True,658.538,65.9494,658.538,1671622093,0,,10,c2668_00009,0


2022-12-21 12:47:36,208	INFO tune.py:777 -- Total run time: 6658.80 seconds (6658.64 seconds for the tuning loop).


In [17]:
result_no_sub.results_df.to_csv("results_without_subgradient_fc.csv")

In [18]:
print(f"""Best loss: {result_no_sub.get_best_config(metric="loss", mode="min")}""")
print(f"""Best accuracy: {result_no_sub.get_best_config(metric="accuracy", mode="max")}""")

Best loss: {'batch_size': 128, 'lr': 0.0027646937592899342, 'alpha': 0, 'beta': 0}
Best accuracy: {'batch_size': 128, 'lr': 0.0027646937592899342, 'alpha': 0, 'beta': 0}


# Further train best models

In [19]:
result_sub_df = pd.read_csv("results_with_subgradient_fc.csv")
result_no_sub_df = pd.read_csv("results_without_subgradient_fc.csv")

In [30]:
ra.ReLU6Function.print_when_zero = True

### Model 1 - with with subradients as hyperparameters

In [63]:
best = result_sub_df.iloc[result_sub_df["loss"].idxmin()]
alpha = best["config/alpha"]
beta = best["config/beta"]
model_sub = rm.FullyConnected(activation=partial(ra.ReLU6, alpha, beta), seed=42).to("cuda")

In [64]:
from torchvision import transforms
from torch.utils.data import DataLoader, random_split

train_set, test_set = ru.default.DATASETS["MNIST"](transforms.Compose([
        transforms.ToTensor(),
    ]))

ru.set_random_seed(42)

train_size = int(len(train_set) * 0.8)
train, val = random_split(
    train_set, [train_size, len(train_set) - train_size])

train_loader = DataLoader(
    train, batch_size=32, shuffle=True, num_workers=8)

val_loader = DataLoader(
    val, batch_size=32, shuffle=False, num_workers=8)

In [65]:
epochs = 30

In [66]:
import math
from ristretto.utils import train_loop, val_loop
import torch.nn.functional as F
from itertools import chain

optimizer = torch.optim.Adam(model_sub.parameters(), lr=0.0009)
criterion = F.cross_entropy

train_metrics = []
val_metrics = []
for epoch in range(epochs):
    print(
        f"---------- Epoch {epoch+1:{math.ceil(math.log10(epochs+1))}d} ----------")
    train_metrics.append(train_loop(
        model_sub, optimizer, criterion, train_loader, verbose=False, dtype=torch.bfloat16))
    val_metrics.append(val_loop(
        model_sub, criterion, val_loader, dtype=torch.bfloat16))

sub_metrics = {
    "train": pd.DataFrame(chain.from_iterable(train_metrics)),
    "validation": pd.DataFrame(chain.from_iterable(val_metrics))
}

---------- Epoch  1 ----------
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 2 items with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Validation
    loss:     2.870e-01
    accuracy: 92.13
---------- Epoch  2 ----------
Found 1 item

### Model 2 - without subgradients as hyperparameters

In [55]:
model_no_sub = rm.FullyConnected(activation=partial(ra.ReLU6, 0, 0), seed=42).to("cuda")

In [56]:
from torchvision import transforms
from torch.utils.data import DataLoader, random_split

train_set, test_set = ru.default.DATASETS["MNIST"](transforms.Compose([
        transforms.ToTensor(),
    ]))

ru.set_random_seed(42)

train_size = int(len(train_set) * 0.8)
train, val = random_split(
    train_set, [train_size, len(train_set) - train_size])

train_loader = DataLoader(
    train, batch_size=128, shuffle=True, num_workers=8)

val_loader = DataLoader(
    val, batch_size=128, shuffle=False, num_workers=8)

In [57]:
epochs = 30

In [58]:
import math
from ristretto.utils import train_loop, val_loop
import torch.nn.functional as F
from itertools import chain

optimizer = torch.optim.Adam(model_no_sub.parameters(), lr=0.0027)
criterion = F.cross_entropy

train_metrics = []
val_metrics = []
for epoch in range(epochs):
    print(
        f"---------- Epoch {epoch+1:{math.ceil(math.log10(epochs+1))}d} ----------")
    train_metrics.append(train_loop(
        model_no_sub, optimizer, criterion, train_loader, verbose=False, dtype=torch.bfloat16))
    val_metrics.append(val_loop(
        model_no_sub, criterion, val_loader, dtype=torch.bfloat16))

no_sub_metrics = {
    "train": pd.DataFrame(chain.from_iterable(train_metrics)),
    "validation": pd.DataFrame(chain.from_iterable(val_metrics))
}

---------- Epoch  1 ----------
Found 2 items with input == 6
Found 1 item with input == 6
Found 2 items with input == 6
Found 1 item with input == 6
Found 3 items with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 2 items with input == 6
Found 2 items with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 2 items with input == 6
Found 2 items with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 1 item with input == 6
Found 2 items with input == 6
Found 1 item with input == 6
Found 4 items with input == 6
Found 1 item with input == 6
Found 2 items with input == 6
Found 2 items with input == 6
Found 1 item with input == 6
Found 4 items with input == 6
Found 2 items with input == 6
Found 3 items with input == 6
Found 1 item with input == 6
Found 2 items with input == 6
Found 2 items with input == 6
Found 2 items with input == 6
Found 1 item with input 